In [4]:
import pandas as pd

df = pd.read_csv("../data/test_logs_clean.csv")

df = df[['id', 'activity']]

print(df.head(3))

def rebuild_text(grp):
    buf = []
    for op in grp['activity']:
        buf.append(op[0])
    return "".join(buf)

operations = (
    df.groupby('id')
        .apply(rebuild_text)
        .reset_index(name='operation')
)

print(operations.head(3))

         id activity
0  0000aaaa    Input
1  0000aaaa    Input
2  2222bbbb    Input
         id operation
0  0000aaaa        II
1  2222bbbb        II
2  4444cccc        II


/var/folders/g1/_6vbzjsx6n5grlp24jr7kjp00000gn/T/ipykernel_42562/2227330183.py:17: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rebuild_text)


In [ ]:
# TF-IDF + SVD for operation feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer(
    analyzer='char',
    # change for test
    ngram_range=(1, 5), 
    max_features=30000,
    dtype=np.float32,
)

X_tfidf = vectorizer.fit_transform(operations['operation'])

tfidf_df = pd.DataFrame(
    X_tfidf.toarray(),
    columns=vectorizer.get_feature_names_out()
)
tfidf_df.insert(0, 'id', operations['id'])

print(tfidf_df.head())

from sklearn.decomposition import TruncatedSVD

n_features = X_tfidf.shape[1]
svdsize = min(64, n_features - 1)  

if svdsize > 0:
    svd = TruncatedSVD(
        n_components=svdsize,
        random_state=42,
        n_iter=7
    )
    X_svd = svd.fit_transform(X_tfidf)
else:
    X_svd = X_tfidf.toarray()
    svdsize = n_features

svd_df = pd.DataFrame(
    X_svd,
    columns=[f'{i:02d}' for i in range(svdsize)]
)
svd_df.insert(0, 'id', operations['id'].values)

print(svd_df.head())
svd_df.to_csv("../data/test_tfidf_operation.csv", index=False)

         id         i        ii
0  0000aaaa  0.894427  0.447214
1  2222bbbb  0.894427  0.447214
2  4444cccc  0.894427  0.447214
         id   00
0  0000aaaa  1.0
1  2222bbbb  1.0
2  4444cccc  1.0


/Users/liyuxiao/miniconda3/envs/nn/lib/python3.12/site-packages/sklearn/decomposition/_truncated_svd.py:273: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var
